The library *pm_executer_6* is the sixth iteration of a geometry simulator where geometries are considered as bicubic Bézier surfaces, as so do both tilt and rotation functions.

In [ ]:
import numpy as np
from pm_executer_6 import * # geometry simulator
from MultipleShanks3 import MultipleShanks # for collision detection
import ToRhino # to produce  ply files to export to rhino
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
# auxiliar functions

def ms(P, radius, resolution=20):
    """Return the coordinates for plotting a sphere centered at (x,y,z)"""
    u, v = np.mgrid[0:2*np.pi:resolution*2j, 0:np.pi:resolution*1j]
    X = radius * np.cos(u)*np.sin(v) + P[0]
    Y = radius * np.sin(u)*np.sin(v) + P[1]
    Z = radius * np.cos(v) + P[2]
    return np.array([X, Y, Z])
def generate_disk_points(P, V, radius, N):
    # Normalize the normal vector
    V = V / np.linalg.norm(V)
    
    # Find two orthogonal vectors in the plane
    # Use a helper vector that is not collinear to V
    if np.allclose(V, [0, 0, 1]) or np.allclose(V, [0, 0, -1]):
        helper_vector = np.array([1, 0, 0])
    else:
        helper_vector = np.array([0, 0, 1])
    
    # First vector in the plane
    U1 = np.cross(V, helper_vector)
    U1 = U1 / np.linalg.norm(U1)
    
    # Second vector in the plane, orthogonal to the first
    U2 = np.cross(V, U1)
    U2 = U2 / np.linalg.norm(U2)
    
    # Generate a grid of points
    theta = np.linspace(0, 2 * np.pi, N)
    r = np.linspace(0, radius, N)
    R, Theta = np.meshgrid(r, theta)  # R, Theta grids
    
    # Flatten the arrays for point calculations
    R = R.flatten()
    Theta = Theta.flatten()
    
    # Compute the points in the disk
    points = P + np.outer(R * np.cos(Theta), U1) + np.outer(R * np.sin(Theta), U2)
    
    return points

In [ ]:
np.random.seed(2) # for reproducibility
Q = np.random.rand(4,4) # surface matrix
G = (1/3)*np.array([[0,0,0,0],
                    [1,1,1,1],
                    [2,2,2,2],
                    [3,3,3,3]]) # gives straight cuts 
# phi = 0.5*np.pi*np.ones((4,4)) # rotation angle fixed to π/2 or "against" the cut
# theta = 0.25*np.pi*np.ones((4,4)) # tilt angle at π/4
np.random.seed(3)
phi =np.random.rand(4,4)
np.random.seed(4)
theta = np.random.rand(4,4)
# theta = 0.15*np.pi*np.ones((4,4)) # tilt angle at π/4
matrices = [G, phi, theta]
R = 1/10 # tool radius
m = 0 # offsetting distance. Set to 0 because we are considering flat-end milling
h = 0.01*R # how much space we leave for the cuts!
number_of_paths = 20 # it is actually the number of level sets of the G function. More paths could be made, but not less
surfaces_resolution = 300 # resolution of the triangulated surfaces for point-line intersection methods
machining_object = MachiningParameters(R, m, Q, 
                                        matrices=matrices,
                                        number_of_paths = number_of_paths,
                                        h = h,
                                        surfaces_resolution=surfaces_resolution)

# G function plot

In [ ]:
superficie = ToRhino.Superficie(matriz=machining_object.matrices[0])
vert_sup, triang_sup = superficie.triangulation()
triang_sup = triang_sup.triangles
U_of_t, V_of_t = machining_object.uv_curves_parametrized()
N = len(machining_object.envelopes)
for n in [N//3, N//2, 2*N//3]:
    u_of_t = U_of_t[n]
    v_of_t = V_of_t[n]
    # curva de nivel funcion
    G_bezier = BezierPatch(control_net=machining_object.matrices[0])
    fun1 = lambda t: G_bezier.Bezier([u_of_t(t), v_of_t(t)])
    tubo = ToRhino.Tubo(curva = fun1, Range=[0,1], R = 0.01)
    # tubo.to_Rhino(name = 'tubo_' + str(n) + '.ply')
    tubo_vertices, tubo_triangulos = tubo.triangulation()

    fun2 = lambda t: np.array([u_of_t(t), v_of_t(t), 0*t])
    tubo2 = ToRhino.Tubo(curva = fun2, Range=[0,1], R = 0.01)
    # tubo2.to_Rhino(name = 'tubo2_' + str(n) + '.ply')
    tubo_vertices2, tubo_triangulos2 = tubo2.triangulation()

    ruled_surface = ToRhino.RuledSurface(curva1 = fun1, curva2 = fun2)
    # ruled_surface.to_Rhino(name = 'ruled_surface_' + str(n) + '.ply')
    ruled_vertices, ruled_triangles = ruled_surface.triangulation()

fig = go.Figure(data=[go.Mesh3d(
        x=np.asarray(vert_sup)[0],
        y=np.asarray(vert_sup)[1],
        z=np.asarray(vert_sup)[2],
        
        i=np.asarray(triang_sup)[:,0],
        j=np.asarray(triang_sup)[:,1],
        k=np.asarray(triang_sup)[:,2],
        name='y',
        showscale=True
    )])
fig.add_mesh3d(
        x=np.asarray(tubo_vertices)[:,0],
        y=np.asarray(tubo_vertices)[:,1],
        z=np.asarray(tubo_vertices)[:,2],
        
        i=np.asarray(tubo_triangulos)[:,0],
        j=np.asarray(tubo_triangulos)[:,1],
        k=np.asarray(tubo_triangulos)[:,2],
        name='y',
        showscale=True)
fig.add_mesh3d(
        x=np.asarray(tubo_vertices2)[:,0],
        y=np.asarray(tubo_vertices2)[:,1],
        z=np.asarray(tubo_vertices2)[:,2],
        
        i=np.asarray(tubo_triangulos2)[:,0],
        j=np.asarray(tubo_triangulos2)[:,1],
        k=np.asarray(tubo_triangulos2)[:,2],
        name='y',
        showscale=True)

fig.add_mesh3d(
        x=np.asarray(ruled_vertices)[:,0],
        y=np.asarray(ruled_vertices)[:,1],
        z=np.asarray(ruled_vertices)[:,2],
        
        i=np.asarray(ruled_triangles)[:,0],
        j=np.asarray(ruled_triangles)[:,1],
        k=np.asarray(ruled_triangles)[:,2],
        name='y',
        showscale=True)
camera = dict(
    up=dict(x=0, y=0, z=1),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=-1, y=-1, z=1)
)
fig.update_layout(
            scene_camera=camera,
            showlegend = False,
            scene=dict(
                xaxis=dict(showgrid=False, visible=False),
                yaxis=dict(showgrid=False, visible=False),
                zaxis=dict(showgrid=False, visible=False),
                aspectmode='data'),
                width = 900,
                height = 900
            )

# Phi

In [ ]:
superficie = ToRhino.Superficie(matriz=machining_object.matrices[1])
vert_sup, triang_sup = superficie.triangulation()
triang_sup = triang_sup.triangles
U_of_t, V_of_t = machining_object.uv_curves_parametrized()
N = len(machining_object.envelopes)
for n in [N//3, N//2, 2*N//3]:
    u_of_t = U_of_t[n]
    v_of_t = V_of_t[n]
    # curva de nivel funcion
    G_bezier = BezierPatch(control_net=machining_object.matrices[1])
    fun1 = lambda t: G_bezier.Bezier([u_of_t(t), v_of_t(t)])
    tubo = ToRhino.Tubo(curva = fun1, Range=[0,1], R = 0.01)
    # tubo.to_Rhino(name = 'tubo_' + str(n) + '.ply')
    tubo_vertices, tubo_triangulos = tubo.triangulation()

    fun2 = lambda t: np.array([u_of_t(t), v_of_t(t), 0*t])
    tubo2 = ToRhino.Tubo(curva = fun2, Range=[0,1], R = 0.01)
    # tubo2.to_Rhino(name = 'tubo2_' + str(n) + '.ply')
    tubo_vertices2, tubo_triangulos2 = tubo2.triangulation()

    ruled_surface = ToRhino.RuledSurface(curva1 = fun1, curva2 = fun2)
    # ruled_surface.to_Rhino(name = 'ruled_surface_' + str(n) + '.ply')
    ruled_vertices, ruled_triangles = ruled_surface.triangulation()

fig = go.Figure(data=[go.Mesh3d(
        x=np.asarray(vert_sup)[0],
        y=np.asarray(vert_sup)[1],
        z=np.asarray(vert_sup)[2],
        
        i=np.asarray(triang_sup)[:,0],
        j=np.asarray(triang_sup)[:,1],
        k=np.asarray(triang_sup)[:,2],
        name='y',
        showscale=True
    )])
fig.add_mesh3d(
        x=np.asarray(tubo_vertices)[:,0],
        y=np.asarray(tubo_vertices)[:,1],
        z=np.asarray(tubo_vertices)[:,2],
        
        i=np.asarray(tubo_triangulos)[:,0],
        j=np.asarray(tubo_triangulos)[:,1],
        k=np.asarray(tubo_triangulos)[:,2],
        name='y',
        showscale=True)
fig.add_mesh3d(
        x=np.asarray(tubo_vertices2)[:,0],
        y=np.asarray(tubo_vertices2)[:,1],
        z=np.asarray(tubo_vertices2)[:,2],
        
        i=np.asarray(tubo_triangulos2)[:,0],
        j=np.asarray(tubo_triangulos2)[:,1],
        k=np.asarray(tubo_triangulos2)[:,2],
        name='y',
        showscale=True)

fig.add_mesh3d(
        x=np.asarray(ruled_vertices)[:,0],
        y=np.asarray(ruled_vertices)[:,1],
        z=np.asarray(ruled_vertices)[:,2],
        
        i=np.asarray(ruled_triangles)[:,0],
        j=np.asarray(ruled_triangles)[:,1],
        k=np.asarray(ruled_triangles)[:,2],
        name='y',
        showscale=True)
camera = dict(
    up=dict(x=0, y=0, z=1),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=-1, y=-1, z=1)
)
fig.update_layout(
            scene_camera=camera,
            showlegend = False,
            scene=dict(
                xaxis=dict(showgrid=False, visible=False),
                yaxis=dict(showgrid=False, visible=False),
                zaxis=dict(showgrid=False, visible=False),
                aspectmode='data'),
                width = 900,
                height = 900
            )

In [ ]:
superficie = ToRhino.Superficie(matriz=machining_object.matrices[2])
vert_sup, triang_sup = superficie.triangulation()
triang_sup = triang_sup.triangles
U_of_t, V_of_t = machining_object.uv_curves_parametrized()
N = len(machining_object.envelopes)
for n in [N//3, N//2, 2*N//3]:
    u_of_t = U_of_t[n]
    v_of_t = V_of_t[n]
    # curva de nivel funcion
    G_bezier = BezierPatch(control_net=machining_object.matrices[2])
    fun1 = lambda t: G_bezier.Bezier([u_of_t(t), v_of_t(t)])
    tubo = ToRhino.Tubo(curva = fun1, Range=[0,1], R = 0.01)
    # tubo.to_Rhino(name = 'tubo_' + str(n) + '.ply')
    tubo_vertices, tubo_triangulos = tubo.triangulation()

    fun2 = lambda t: np.array([u_of_t(t), v_of_t(t), 0*t])
    tubo2 = ToRhino.Tubo(curva = fun2, Range=[0,1], R = 0.01)
    # tubo2.to_Rhino(name = 'tubo2_' + str(n) + '.ply')
    tubo_vertices2, tubo_triangulos2 = tubo2.triangulation()

    ruled_surface = ToRhino.RuledSurface(curva1 = fun1, curva2 = fun2)
    # ruled_surface.to_Rhino(name = 'ruled_surface_' + str(n) + '.ply')
    ruled_vertices, ruled_triangles = ruled_surface.triangulation()

fig = go.Figure(data=[go.Mesh3d(
        x=np.asarray(vert_sup)[0],
        y=np.asarray(vert_sup)[1],
        z=np.asarray(vert_sup)[2],
        
        i=np.asarray(triang_sup)[:,0],
        j=np.asarray(triang_sup)[:,1],
        k=np.asarray(triang_sup)[:,2],
        name='y',
        showscale=True
    )])
fig.add_mesh3d(
        x=np.asarray(tubo_vertices)[:,0],
        y=np.asarray(tubo_vertices)[:,1],
        z=np.asarray(tubo_vertices)[:,2],
        
        i=np.asarray(tubo_triangulos)[:,0],
        j=np.asarray(tubo_triangulos)[:,1],
        k=np.asarray(tubo_triangulos)[:,2],
        name='y',
        showscale=True)
fig.add_mesh3d(
        x=np.asarray(tubo_vertices2)[:,0],
        y=np.asarray(tubo_vertices2)[:,1],
        z=np.asarray(tubo_vertices2)[:,2],
        
        i=np.asarray(tubo_triangulos2)[:,0],
        j=np.asarray(tubo_triangulos2)[:,1],
        k=np.asarray(tubo_triangulos2)[:,2],
        name='y',
        showscale=True)

fig.add_mesh3d(
        x=np.asarray(ruled_vertices)[:,0],
        y=np.asarray(ruled_vertices)[:,1],
        z=np.asarray(ruled_vertices)[:,2],
        
        i=np.asarray(ruled_triangles)[:,0],
        j=np.asarray(ruled_triangles)[:,1],
        k=np.asarray(ruled_triangles)[:,2],
        name='y',
        showscale=True)
camera = dict(
    up=dict(x=0, y=0, z=1),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=-1, y=-1, z=1)
)
fig.update_layout(
            scene_camera=camera,
            showlegend = False,
            scene=dict(
                xaxis=dict(showgrid=False, visible=False),
                yaxis=dict(showgrid=False, visible=False),
                zaxis=dict(showgrid=False, visible=False),
                aspectmode='data'),
                width = 900,
                height = 900
            )

# Surface plot with contact curves and a cylinder

In [ ]:
x = np.linspace(0,1,100)
y = np.linspace(0,1,100)
X, Y = np.meshgrid(x, y)
puntos_sup = machining_object.surface.function(np.array([X.reshape(-1),Y.reshape(-1)]))
puntos_sup = puntos_sup.reshape(3,100,100)

# Assuming you already have a surface plot:
lighting_effects = dict(ambient=0.4, diffuse=0.5, roughness = 0.9, specular=0.1, fresnel=2.5)
# trace = go.Surface(z=Z1, colorscale='Viridis', lighting=lighting_effects)
fig = go.Figure()
fig.add_surface(x = puntos_sup[0], y=puntos_sup[1], z = puntos_sup[2],colorscale='Blues', lighting=lighting_effects, showscale=False)

# add the contact curve
envolvente= machining_object.envelopes[len(machining_object.envelopes)//2 ]
X,Y,Z = envolvente.curve(np.linspace(0,1,1000))
fig.add_scatter3d(x = X, y=Y, z=Z, mode = 'markers' ,marker=dict(
                                    size= 2,
                                    color ='black',
                                    opacity = 1
                                ))

# add the rest of the contact paths but thiner
for envelope  in machining_object.envelopes:
    X,Y,Z = envelope.curve(np.linspace(0,1,100))
    fig.add_scatter3d(x = X, y=Y, z=Z, mode = 'markers+lines' ,marker=dict(
                                        size= 1,
                                        color ='black',
                                        opacity = 1
                                    ))


# add the circle at time t0
t0 = 0.5
X,Y,Z = envolvente.value_at(s = np.linspace(0, 2*np.pi, 1000), t = t0, method = 'fixed_arc').T
fig.add_scatter3d(x = X, y=Y, z=Z, mode = 'markers' ,marker=dict(
                                    size= 4,
                                    color ='purple',
                                    opacity = 1
                                ))

# add the cylinder
X,Y,Z = envolvente.cilindro_at_t(t = t0).T
fig.add_surface(x = X, y=Y, z = Z,colorscale='algae', lighting=lighting_effects, opacity = 0.2, showscale=False)

# add the shank
shank = envolvente.shank_at_t(t = t0)
X,Y,Z =np.linspace(shank[0], shank[1], 1000).T
fig.add_scatter3d(x = X, y=Y, z=Z, mode = 'markers+lines' ,marker=dict(
                                    size= 4,
                                    color ='black',
                                    opacity = 1
                                ))

# add disk
P = shank[0]
V = shank[0]-shank[1]
X,Y,Z = generate_disk_points(P, V, machining_object.R, 100).T
fig.add_mesh3d(x = X, y=Y, z = Z,colorscale='algae', lighting=lighting_effects, opacity = 0.8)


# add contact point
X,Y,Z = np.atleast_2d([envolvente.curve(t0)]).T
fig.add_scatter3d(x = X, y=Y, z=Z, mode = 'markers' ,marker=dict(
                                size= 6,
                                color ='brown',
                                opacity = 1
                            ))

camera = dict(
    up=dict(x=0, y=0, z=1),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=-1, y=-1, z=1)
)
fig.update_layout(
            scene_camera=camera,
            showlegend = False,
            scene=dict(
                xaxis=dict(showgrid=False, visible=False),
                yaxis=dict(showgrid=False, visible=False),
                zaxis=dict(showgrid=False, visible=False),
                aspectmode='data'),
                width = 900,
                height = 900
            )



# Plot of surface and envelopes

In [ ]:
machining_object.number_of_paths = 5
x = np.linspace(0,1,100)
y = np.linspace(0,1,100)
X, Y = np.meshgrid(x, y)
puntos_sup = machining_object.surface.function(np.array([X.reshape(-1),Y.reshape(-1)]))
puntos_sup = puntos_sup.reshape(3,100,100)

# Assuming you already have a surface plot:
lighting_effects = dict(ambient=0.4, diffuse=0.5, roughness = 0.9, specular=0.1, fresnel=2.5)
# trace = go.Surface(z=Z1, colorscale='Viridis', lighting=lighting_effects)
fig = go.Figure()
fig.add_surface(x = puntos_sup[0], y=puntos_sup[1], z = puntos_sup[2],colorscale='Blues', lighting=lighting_effects, showscale=False)

# add the contact curve
envolvente= machining_object.envelopes[len(machining_object.envelopes)//2 ]
X,Y,Z = envolvente.curve(np.linspace(0,1,1000))
fig.add_scatter3d(x = X, y=Y, z=Z, mode = 'markers' ,marker=dict(
                                    size= 2,
                                    color ='black',
                                    opacity = 1
                                ))

# add envelope plot
X,Y,Z = envolvente.value_at(s = np.linspace(0,2*np.pi,100), t = np.linspace(0,1,100), method = 'whole_envelope').T
fig.add_surface(x = X, y=Y, z = Z,colorscale='gray', lighting=lighting_effects, showscale=False)

# add the rest of the contact paths but thiner
for envelope  in machining_object.envelopes:
    X,Y,Z = envelope.curve(np.linspace(0,1,100))
    fig.add_scatter3d(x = X, y=Y, z=Z, mode = 'markers+lines' ,marker=dict(
                                        size= 1,
                                        color ='black',
                                        opacity = 1
                                    ))




# add the circle at time t0
t0 = 0.5
X,Y,Z = envolvente.value_at(s = np.linspace(0, 2*np.pi, 1000), t = t0, method = 'fixed_arc').T
fig.add_scatter3d(x = X, y=Y, z=Z, mode = 'markers' ,marker=dict(
                                    size= 4,
                                    color ='purple',
                                    opacity = 1
                                ))

# add the cylinder
X,Y,Z = envolvente.cilindro_at_t(t = t0).T
fig.add_surface(x = X, y=Y, z = Z,colorscale='algae', lighting=lighting_effects, opacity = 0.2, showscale=False)

# add the shank
shank = envolvente.shank_at_t(t = t0)
X,Y,Z =np.linspace(shank[0], shank[1], 1000).T
fig.add_scatter3d(x = X, y=Y, z=Z, mode = 'markers+lines' ,marker=dict(
                                    size= 4,
                                    color ='black',
                                    opacity = 1
                                ))

# add disk
P = shank[0]
V = shank[0]-shank[1]
X,Y,Z = generate_disk_points(P, V, machining_object.R, 100).T
fig.add_mesh3d(x = X, y=Y, z = Z,colorscale='algae', lighting=lighting_effects, opacity = 0.8)


# add contact point
X,Y,Z = np.atleast_2d([envolvente.curve(t0)]).T
fig.add_scatter3d(x = X, y=Y, z=Z, mode = 'markers' ,marker=dict(
                                size= 6,
                                color ='brown',
                                opacity = 1
                            ))

camera = dict(
    up=dict(x=0, y=0, z=1),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=-1, y=-1, z=1)
)
fig.update_layout(
            scene_camera=camera,
            showlegend = False,
            scene=dict(
                xaxis=dict(showgrid=False, visible=False),
                yaxis=dict(showgrid=False, visible=False),
                zaxis=dict(showgrid=False, visible=False),
                aspectmode='data'),
                width = 900,
                height = 900
            )

In [ ]:
x = np.linspace(0,1,100)
y = np.linspace(0,1,100)
X, Y = np.meshgrid(x, y)
puntos_sup = machining_object.surface.function(np.array([X.reshape(-1),Y.reshape(-1)]))
puntos_sup = puntos_sup.reshape(3,100,100)

# Assuming you already have a surface plot:
lighting_effects = dict(ambient=0.4, diffuse=0.5, roughness = 0.9, specular=0.1, fresnel=2.5)
# trace = go.Surface(z=Z1, colorscale='Viridis', lighting=lighting_effects)
fig = go.Figure()
fig.add_surface(x = puntos_sup[0], y=puntos_sup[1], z = puntos_sup[2],colorscale='Blues', lighting=lighting_effects, showscale=False)

# add the contact curve
envolvente= machining_object.envelopes[len(machining_object.envelopes)//2 ]
X,Y,Z = envolvente.curve(np.linspace(0,1,1000))
fig.add_scatter3d(x = X, y=Y, z=Z, mode = 'markers' ,marker=dict(
                                    size= 2,
                                    color ='black',
                                    opacity = 1
                                ))

# add envelope plot
# X,Y,Z = envolvente.value_at(s = np.linspace(0,2*np.pi,100), t = np.linspace(0,1,100), method = 'whole_envelope').T
# fig.add_surface(x = X, y=Y, z = Z,colorscale='gray', lighting=lighting_effects, showscale=False)

discretized_envelopes = np.array([
    env.value_at(machining_object.angles, machining_object.times, method='whole_envelope').reshape(-1,3) 
    for env in machining_object.envelopes])
X,Y,Z = discretized_envelopes.T
fig.add_surface(x = X, y=Y, z = Z,colorscale='gray', lighting=lighting_effects, showscale=False)

# add the rest of the contact paths but thiner
for envelope  in machining_object.envelopes:
    X,Y,Z = envelope.curve(np.linspace(0,1,100))
    fig.add_scatter3d(x = X, y=Y, z=Z, mode = 'markers+lines' ,marker=dict(
                                        size= 1,
                                        color ='black',
                                        opacity = 1
                                    ))




# add the circle at time t0
t0 = 0.5
X,Y,Z = envolvente.value_at(s = np.linspace(0, 2*np.pi, 1000), t = t0, method = 'fixed_arc').T
fig.add_scatter3d(x = X, y=Y, z=Z, mode = 'markers' ,marker=dict(
                                    size= 4,
                                    color ='purple',
                                    opacity = 1
                                ))

# add the cylinder
X,Y,Z = envolvente.cilindro_at_t(t = t0).T
fig.add_surface(x = X, y=Y, z = Z,colorscale='algae', lighting=lighting_effects, opacity = 0.2, showscale=False)

# add the shank
shank = envolvente.shank_at_t(t = t0)
X,Y,Z =np.linspace(shank[0], shank[1], 1000).T
fig.add_scatter3d(x = X, y=Y, z=Z, mode = 'markers+lines' ,marker=dict(
                                    size= 4,
                                    color ='black',
                                    opacity = 1
                                ))

# add disk
P = shank[0]
V = shank[0]-shank[1]
X,Y,Z = generate_disk_points(P, V, machining_object.R, 100).T
fig.add_mesh3d(x = X, y=Y, z = Z,colorscale='algae', lighting=lighting_effects, opacity = 0.8)


# add contact point
X,Y,Z = np.atleast_2d([envolvente.curve(t0)]).T
fig.add_scatter3d(x = X, y=Y, z=Z, mode = 'markers' ,marker=dict(
                                size= 6,
                                color ='brown',
                                opacity = 1
                            ))

camera = dict(
    up=dict(x=0, y=0, z=1),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=-1, y=-1, z=1)
)
fig.update_layout(
            scene_camera=camera,
            showlegend = False,
            scene=dict(
                xaxis=dict(showgrid=False, visible=False),
                yaxis=dict(showgrid=False, visible=False),
                zaxis=dict(showgrid=False, visible=False),
                aspectmode='data'),
                width = 900,
                height = 900
            )

In [ ]:
def cilindro(bottom, top, R):
    translation_vector = top - bottom
    unit_trans_vect = translation_vector/np.linalg.norm(translation_vector)
    v1 = np.array([-unit_trans_vect[1], unit_trans_vect[0], 0])
    v1 = v1/np.linalg.norm(v1)
    v2 = np.cross(v1, unit_trans_vect)
    # tenemos los vectores, ahora hay que sacarle los puntos al circulo

    centro = bottom
    circulo = centro + R*np.array([np.cos(s)*v1 + np.sin(s)*v2 for s in np.linspace(0,2*np.pi, 50)])
    cylinder = np.array([ i*translation_vector + circulo for i in np.linspace(0,1,50) ])
    return cylinder

def sphere(P, radius, resolution=20):
    """Return the coordinates for plotting a sphere centered at (x,y,z)"""
    u, v = np.mgrid[0:2*np.pi:resolution*2j, 0:np.pi:resolution*1j]
    X = radius * np.cos(u)*np.sin(v) + P[0]
    Y = radius * np.sin(u)*np.sin(v) + P[1]
    Z = radius * np.cos(v) + P[2]
    return np.array([X, Y, Z])

def collision_score(machining_object, which_MS = 2):
    '''Detects if machining_object is colliding.'''
    mesh, scene = pickable_to_mesh([machining_object.surface_vertices, machining_object.surface_triangles])
    if which_MS ==2:

        shanksObject = MS2.MultipleShanks(
                        set_of_shanks=machining_object.discrete_shanks_vectorized(n_shanks = 300),
                        mesh=mesh, 
                        centroids = machining_object.surface_centroids,
                        scene = scene,
                        R = machining_object.R,
                        relative_step = 0.1)
        iteracion, information_matrix = shanksObject.optimal_multishank_collision_without_gaps()
    elif which_MS ==3:
        shanksObject = MultipleShanks(
                        set_of_shanks=machining_object.discrete_shanks_vectorized(n_shanks = 300),
                        mesh=mesh, 
                        centroids = machining_object.surface_centroids,
                        scene = scene,
                        R = machining_object.R)
        iteracion, information_matrix = shanksObject.collision_detection_no_gaps()
    else:
        print('Wrong MS')
    is_collision = np.any(information_matrix[:, 0])
    if is_collision:
        collision_score = 1e6
    else:
        collision_score = 0
    return (collision_score, information_matrix)
def objective_function_information(solution, K):
    machining_object.matrices = solution
    errors_per_triangle = machining_object.error_measure_per_triangle_open3d_bis()
    mask_gauging_values = errors_per_triangle < 0
    errors_per_triangle[mask_gauging_values] = -100*errors_per_triangle[mask_gauging_values]
    col_score = collision_score(machining_object, which_MS = 3)[0]
    n_passes = len(machining_object.envelopes)
    curves_length = np.array([env.length_of_contact_line for env in machining_object.envelopes]).mean()
    machining_error_normalized = errors_per_triangle.sum()/(machining_object.machining_tolerance * len(machining_object.surface_centroids))
    max_scaler = np.min([10, errors_per_triangle.max()])
    machining_time_normalized = curves_length*n_passes / (2*np.sqrt(2)*number_of_paths)
    score = col_score + K*machining_error_normalized*np.exp(max_scaler) + (max_K-K)*machining_time_normalized
    # score = col_score + 1000*errors_per_triangle.sum() * np.exp(max_scaler)
    return (score, col_score, errors_per_triangle.min(), errors_per_triangle.max(), curves_length)

def objective_function(solution, K):
    '''Objective function to optimize'''
    return objective_function_information(solution, K)[0]

def vector_to_segment(vector, segment):
    '''sends each element of vector to its corresponding value inside segment by linear interpolation'''
    a,b = segment
    return (1-vector)*a + vector*b
def transform(scaled_solution):

    # Limites de las variables que forman la solucion
    bound_phi   = [0, np.pi]
    bound_theta = [-0.25*np.pi, 0.25*np.pi]
    bound_G     = [0, 1] # not sure about this one but should work
    #descomposicion y reshaping
    n_control_points = 4
    sol_G = vector_to_segment(scaled_solution[: 16], bound_G).reshape(n_control_points,n_control_points)
    sol_phi = vector_to_segment(scaled_solution[16 : 32], bound_phi).reshape(n_control_points,n_control_points)
    sol_theta = vector_to_segment(scaled_solution[32:], bound_theta).reshape(n_control_points,n_control_points)
    # desescalado de las matrices y reestructurado
    real_solution = [sol_G, sol_phi, sol_theta]#, number_of_paths]

    return real_solution

In [ ]:
def heatmap(distances = False, colorscale = 'turbo_r'):
    if not distances:
        distances =  machining_object.error_measure_per_triangle_open3d_bis()
        col_score, information_matrix = collision_score( machining_object, which_MS=3)
    if col_score >0:
        colorscale = 'RdBu'
    cmin, cmax =  machining_object.machining_tolerance*np.array([-1,1])
    surface_mesh, scene = pickable_to_mesh([ machining_object.surface_vertices,  machining_object.surface_triangles])
    distances_adjusted_to_tolerances = distances.copy()
    distances_adjusted_to_tolerances[distances_adjusted_to_tolerances > cmax] = cmax # set undercut
    distances_adjusted_to_tolerances[distances_adjusted_to_tolerances < cmin] = cmin # set overcut
    # Create the figure
    fig = go.Figure(data=[go.Mesh3d(
        x=np.asarray(surface_mesh.vertices)[:,0],
        y=np.asarray(surface_mesh.vertices)[:,1],
        z=np.asarray(surface_mesh.vertices)[:,2],
        colorscale = colorscale,
        cmin=cmin,
        cmax=cmax,
        intensity=distances_adjusted_to_tolerances,
        intensitymode = 'cell', 
        i=np.asarray(surface_mesh.triangles)[:,0],
        j=np.asarray(surface_mesh.triangles)[:,1],
        k=np.asarray(surface_mesh.triangles)[:,2],
        name='y',
        showscale=True
    )])
    
    for env in  machining_object.envelopes:
        # contact curves
        contact_points = env.curve(np.linspace(0,1,100))
        x,y,z = contact_points
        fig.add_scatter3d(x = x, y=y, z=z, marker = dict(size = 1, color = 'black'), showlegend=False)

    if col_score > 0:
        # if there is a collision, render only the colliding shanks, their cylinders and their stuff in general
        colliding_shanks = information_matrix[information_matrix[:, 0] == True]
        for shank in colliding_shanks:
            points_in_shank = np.array(shank[5])
            footpoints = np.array(shank[6])
            x,y,z= np.array([shank[2], shank[3]]).T # bottom top
            fig.add_scatter3d(x=x, y=y, z=z, marker=dict(size = 1, color = 'black'))
            for point, footpoint in zip(points_in_shank, footpoints):
                X,Y,Z = np.array([point, footpoint]).T
                fig.add_scatter3d(x = X, y=Y, z=Z, mode = 'markers+lines' ,marker=dict(
                                                size= 4,
                                                color ='orange',
                                                opacity = 1
                                            ))
                X,Y,Z = sphere(point, np.linalg.norm(footpoint-point) -  machining_object.R, resolution=100)
                fig.add_surface(x = X, y=Y, z = Z,colorscale='oranges', opacity = 0.6, showscale = False)
            bottom, top = shank[2], shank[3]
            cylinder = cilindro(bottom, top,  machining_object.R)
            X,Y,Z = cylinder.T
            fig.add_surface(x = X, y=Y, z = Z,colorscale='algae', opacity = 0.5, showscale=False)
    else:
        # otherwise, just plot the shanks, but all of them
        pass
        # for shank in information_matrix[0::30]:
        #     x,y,z= np.array([shank[2], shank[3]]).T # bottom top
        #     fig.add_scatter3d(x=x, y=y, z=z, marker=dict(size = 1, color = 'black'))



    fig.update_layout(
        showlegend = True,
        scene=dict(
            xaxis=dict(showgrid=False, visible=False),
            yaxis=dict(showgrid=False, visible=False),
            zaxis=dict(showgrid=False, visible=False),
            aspectmode='data'),
            width = 1600,
            height = 900
        )


    return fig

In [ ]:
fig = heatmap()
fig.show()

In [ ]:
x = np.linspace(0,1,100)
y = np.linspace(0,1,100)
X, Y = np.meshgrid(x, y)
puntos_sup = machining_object.surface.function(np.array([X.reshape(-1),Y.reshape(-1)]))
puntos_sup = puntos_sup.reshape(3,100,100)

# Assuming you already have a surface plot:
lighting_effects = dict(ambient=0.4, diffuse=0.5, roughness = 0.9, specular=0.1, fresnel=2.5)
# trace = go.Surface(z=Z1, colorscale='Viridis', lighting=lighting_effects)
fig = go.Figure()
fig.add_surface(x = puntos_sup[0], y=puntos_sup[1], z = puntos_sup[2],colorscale='Blues', lighting=lighting_effects, showscale=False)

# add the contact curve
envolvente= machining_object.envelopes[len(machining_object.envelopes)//2 ]
X,Y,Z = envolvente.curve(np.linspace(0,1,1000))
fig.add_scatter3d(x = X, y=Y, z=Z, mode = 'markers' ,marker=dict(
                                    size= 2,
                                    color ='black',
                                    opacity = 1
                                ))

# add the circle at time t0
t0 = 0.5
X,Y,Z = envolvente.value_at(s = np.linspace(0, 2*np.pi, 1000), t = t0, method = 'fixed_arc').T
fig.add_scatter3d(x = X, y=Y, z=Z, mode = 'markers' ,marker=dict(
                                    size= 4,
                                    color ='purple',
                                    opacity = 1
                                ))

# add the cylinder
X,Y,Z = envolvente.cilindro_at_t(t = t0).T
fig.add_surface(x = X, y=Y, z = Z,colorscale='algae', lighting=lighting_effects, opacity = 0.2, showscale=False)

# add the shank
shank = envolvente.shank_at_t(t = t0)
X,Y,Z =np.linspace(shank[0], shank[1], 1000).T
fig.add_scatter3d(x = X, y=Y, z=Z, mode = 'markers+lines' ,marker=dict(
                                    size= 4,
                                    color ='black',
                                    opacity = 1
                                ))

# add disk
P = shank[0]
V = shank[0]-shank[1]
X,Y,Z = generate_disk_points(P, V, machining_object.R, 100).T
fig.add_mesh3d(x = X, y=Y, z = Z,colorscale='algae', lighting=lighting_effects, opacity = 0.8)

# add points in shanks and footpoints
mesh, scene = pickable_to_mesh([machining_object.surface_vertices, machining_object.surface_triangles])
shanksObject = MultipleShanks(
                set_of_shanks=np.array([shank]),
                mesh=mesh, 
                centroids = machining_object.surface_centroids,
                scene = scene,
                R = machining_object.R)
iteracion, information_matrix = shanksObject.collision_detection_no_gaps()

index_of_is_shank_colliding = 0
index_of_shank_index = 1
index_of_bottom = 2
index_of_top = 3
index_of_list_of_points_in_shank = 4
index_of_list_of_footpoints = 5
index_of_list_of_safe_distances = 6
index_of_collision_indices = 7
index_of_list_of_iterations = 8


points_in_shank = np.array(information_matrix[0][index_of_list_of_points_in_shank])
footpoints = np.array(information_matrix[0][index_of_list_of_footpoints])
for point, footpoint in zip(points_in_shank, footpoints):
    X,Y,Z = np.array([point, footpoint]).T
    fig.add_scatter3d(x = X, y=Y, z=Z, mode = 'markers+lines' ,marker=dict(
                                    size= 4,
                                    color ='orange',
                                    opacity = 1
                                ))
    X,Y,Z = ms(point, np.linalg.norm(footpoint-point) - machining_object.R, resolution=100)
    fig.add_surface(x = X, y=Y, z = Z,colorscale='oranges', lighting=lighting_effects, opacity = 0.6, showscale = False)

# add contact point
X,Y,Z = np.atleast_2d([envolvente.curve(t0)]).T
fig.add_scatter3d(x = X, y=Y, z=Z, mode = 'markers' ,marker=dict(
                                size= 6,
                                color ='brown',
                                opacity = 1
                            ))
camera = dict(
    up=dict(x=0, y=0, z=1),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=-1, y=-1, z=1)
)
fig.update_layout(
            scene_camera=camera,
            showlegend = False,
            scene=dict(
                xaxis=dict(showgrid=False, visible=False),
                yaxis=dict(showgrid=False, visible=False),
                zaxis=dict(showgrid=False, visible=False),
                aspectmode='data'),
                width = 900,
                height = 900
            )


In [ ]:
np.array([env.length_of_contact_line for env in machining_object.envelopes])

In [ ]:
machining_object.number_of_paths

In [ ]:
machining_object.number_of_paths = 10 
np.array([env.length_of_contact_line for env in machining_object.envelopes])

In [ ]:
machining_object.number_of_paths